# Evaluate model endpoints using Prompt Flow Eval APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate response from MaaS endpoints deployed on Azure AI Platform, as well as external model endpoints such as model deployed on HuggingFace platform.

This guide uses Python Class as a target to evaluate results. 

In [ ]:
%pip install promptflow-evals
%pip install promptflow-azure


In [ ]:
%pip install promptflow-evals
%pip install promptflow-azure

import pandas as pd
import os

from pprint import pprint
from pathlib import Path

import json
import requests

from typing import List, Tuple, TypedDict


Please provide Azure AI Project details so that traces and eval results are pushing in the project. 

In [ ]:
azure_ai_project = {
    "subscription_id": "***",
    "resource_group_name": "***",
    "project_name": "***"
}


For simplicity, we have provided endpoints and keys in the code below. 
We do recommend keeping these endpoints and keys in env variables. 

In [ ]:
env_var = {
    "tiny_llama" : {
        "endpoint" : "https://api-inference.huggingface.co/models/TinyLlama/TinyLlama-1.1B-Chat-v1.0/v1/chat/completions",
	    "key" : "***",
    },
    "phi3_mini_serverless" : {
        "endpoint" : "https://Phi-3-mini-4k-instruct-rqvel.eastus2.models.ai.azure.com/v1/chat/completions",
	    "key" : "***",
    },
    "gpt2" : {
        "endpoint" : "https://api-inference.huggingface.co/models/openai-community/gpt2",
	    "key" : "***",
    },
    "mistral7b" : {
        "endpoint" : "https://mistral-7b-east1092381.eastus2.inference.ml.azure.com/chat/completions",
	    "key" : "***",
    },
}


Following code reads Json file "data.jsonl" which contains inputs of the Target function. 

In [ ]:
df = pd.read_json("testdata/data.jsonl", lines=True)
print(df.head())

Following code runs Evaluate API and uses Content Safety Evaluator to evaluate results.
Test data is provided in json file 'data.jsonl' for Target Function. 
It contains 'question' and the model type. 
Target function uses the questions to call specific endpoints and retrive answer from response to evaluate using Evaluate API from Promoptflow SDK. 

In [ ]:
from app_target import ModelEndpoints
import random

from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import ContentSafetyEvaluator, RelevanceEvaluator, CoherenceEvaluator

configuration = AzureOpenAIModelConfiguration(
    azure_endpoint="https://ai-***.openai.azure.com",
    api_key="***",
    api_version="2023-03-15-preview",
    azure_deployment="gpt-35-turbo",
)


content_safety_evaluator = ContentSafetyEvaluator(project_scope=azure_ai_project)
relevance_evaluator = RelevanceEvaluator(model_config=configuration)
coherence_evaluator = CoherenceEvaluator(model_config=configuration)

models = ["tiny_llama", "phi3_mini_serverless", "gpt2", "mistral7b"]

path = os.path.join(os.getcwd(), "testdata", "data.jsonl")

for model in models:
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        azure_ai_project=azure_ai_project,
        evaluation_name="Eval-Run-"+str(randomNum)+"-"+model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators = {
            "content_safety": content_safety_evaluator,
            "coherence": coherence_evaluator,
            "relevance": relevance_evaluator,
        },
        evaluator_config={
            "content_safety": {
                "question": "${data.question}",
                "answer": "${target.answer}"
            },
            "coherence": {
                "answer": "${target.answer}",
                "question": "${data.question}"
            },
            "relevance": {
                "answer": "${target.answer}",
                "context": "${data.context}",
                "question": "${data.question}"
            }
        })
    results